In [ ]:
import sagemaker
from sagemaker.model import Model
from sagemaker.async_inference.async_inference_config import AsyncInferenceConfig
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer

async_config = AsyncInferenceConfig(
    max_concurrent_invocations_per_instance=1,
    output_path="s3://dev-archery-e-coach-bucket/async-results/"
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
bucket_name = "dev-archery-e-coach-bucket"

In [ ]:
# Crear directorio temporal
!mkdir -p model_package/code

# Copiar archivos de código
!cp code/inference.py model_package/code/
!cp code/utils.py model_package/code/
!cp requirements.txt model_package/code/
!cp code/config.json model_package/code/

# Crear el archivo tar.gz
!cd model_package && tar -czvf ../model.tar.gz . 

# Limpiar
!rm -rf model_package

In [ ]:
# Subir model.tar.gz a S3
sagemaker_session = sagemaker.Session()

model_data = sagemaker_session.upload_data(
    path='model.tar.gz',
    bucket=bucket_name,
    key_prefix='pose-analysis-model'
)

In [4]:
model = Model(
    image_uri="844377604406.dkr.ecr.us-east-1.amazonaws.com/archery-e-coach-pose-estimation-custom-image",
    model_data="s3://dev-archery-e-coach-bucket/pose-analysis-model/model.tar.gz",
    role="arn:aws:iam::844377604406:role/service-role/AmazonSageMaker-ExecutionRole-20230510T103029",
    env={'OUTPUT_BUCKET': bucket_name},
    predictor_cls=None
)


In [ ]:
async_predictor = model.deploy(
    endpoint_name="archery-e-coach-async-endpoint",
    serializer=JSONSerializer,
    deserializer=JSONDeserializer,
    instance_type="ml.m5.large",
    initial_instance_count=1,
    async_inference_config=async_config,
)

# Invoke

In [4]:
import boto3
import time
import json

runtime = boto3.client('sagemaker-runtime')

input_data = {
    "bucket": "dev-archery-e-coach-bucket",
    "key": "videos/in/men2.mp4"
}

In [5]:
response = runtime.invoke_endpoint_async(
    EndpointName='archery-e-coach-async-endpoint',
    InputLocation="s3://dev-archery-e-coach-bucket/videos/2af83daf-c5bb-497b-bb77-08ee94319a30.json",
    ContentType='application/json',
    Accept='application/json'
)

In [6]:
response

{'ResponseMetadata': {'RequestId': '529ff682-2c4e-4e69-bd8e-861084c4211a',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'x-amzn-requestid': '529ff682-2c4e-4e69-bd8e-861084c4211a',
   'x-amzn-sagemaker-outputlocation': 's3://dev-archery-e-coach-bucket/async-results/e01e145b-b4c8-49a1-bf26-4dc5cc798894.out',
   'x-amzn-sagemaker-failurelocation': 's3://sagemaker-us-east-1-844377604406/async-endpoint-failures/archery-e-coach-pose-estimation-custom--2025-02-1738944701-0bc6/e01e145b-b4c8-49a1-bf26-4dc5cc798894-error.out',
   'date': 'Sat, 08 Feb 2025 00:59:28 GMT',
   'content-type': 'application/json',
   'content-length': '54',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'OutputLocation': 's3://dev-archery-e-coach-bucket/async-results/e01e145b-b4c8-49a1-bf26-4dc5cc798894.out',
 'FailureLocation': 's3://sagemaker-us-east-1-844377604406/async-endpoint-failures/archery-e-coach-pose-estimation-custom--2025-02-1738944701-0bc6/e01e145b-b4c8-49a1-bf26-4dc5cc798894-error.out',
 'Inferen

In [14]:

sagemaker = boto3.client('sagemaker')